In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,512 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,734 kB]
Get:13 https://developer.download.nvidia.com/compute/cuda/

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame.
# Read the CSV data from the URL into a Spark DataFrame
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Create a temporary view of the DataFrame
home_sales_df.createOrReplaceTempView("home_sales")

# Verify the temporary view creation
spark.sql("SHOW TABLES").show()


+---------+----------+-----------+
|namespace| tableName|isTemporary|
+---------+----------+-----------+
|         |home_sales|       true|
+---------+----------+-----------+



In [5]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
# Query to calculate the average price for a four-bedroom house sold per year
avg_price_four_bedrooms = spark.sql("""
    SELECT
        YEAR(date) AS year_sold,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date)
    ORDER BY year_sold
""")

# Show the result
avg_price_four_bedrooms.show()



+---------+---------+
|year_sold|avg_price|
+---------+---------+
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
|     2022|296363.88|
+---------+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# Query to calculate the average price for homes with 3 bedrooms and 3 bathrooms per year built
avg_price_three_bed_bath = spark.sql("""
    SELECT
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY year_built
""")

# Show the result
avg_price_three_bed_bath.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [7]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
# Query to calculate the average price of homes meeting the specified conditions
avg_price_specific_homes = spark.sql("""
    SELECT
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3
      AND bathrooms = 3
      AND floors = 2
      AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY year_built
""")

# Show the result
avg_price_specific_homes.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [8]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
import time

# Start the timer
start_time = time.time()

# Query to calculate the average price of a home per "view" rating
avg_price_per_view = spark.sql("""
    SELECT
        view AS view_rating,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view_rating DESC
""")

# Show the results
avg_price_per_view.show()

# End the timer and print runtime
print("--- %s seconds ---" % (time.time() - start_time))


+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 0.9246087074279785 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.

spark.sql("CACHE TABLE home_sales")

# Verify if the table is cached
print("Is 'home_sales' table cached? ", spark.catalog.isCached("home_sales"))


Is 'home_sales' table cached?  True


In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

# Start the timer
start_time = time.time()

# Query to calculate the average price of a home per "view" rating using the cached table
avg_price_per_view_cached = spark.sql("""
    SELECT
        view AS view_rating,
        ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view_rating DESC
""")

# Show the results
avg_price_per_view_cached.show()

# End the timer and print runtime
print("--- %s seconds ---" % (time.time() - start_time))



+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 0.49667835235595703 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

home_sales_df.write.partitionBy("date_built").parquet("home_sales_partitioned")

print("Data has been successfully partitioned and saved as Parquet.")


Data has been successfully partitioned and saved as Parquet.


In [13]:
# 11. Read the parquet formatted data.

partitioned_home_sales_df = spark.read.parquet("home_sales_partitioned")

# Show the DataFrame schema and a few rows to verify
partitioned_home_sales_df.printSchema()
partitioned_home_sales_df.show(5)



root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- date_built: integer (nullable = true)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|

In [14]:
# 12. Create a temporary table for the parquet data.

partitioned_home_sales_df.createOrReplaceTempView("partitioned_home_sales")

# Verify that the table is created
spark.sql("SHOW TABLES").show()


+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|         |          home_sales|       true|
|         |partitioned_home_...|       true|
+---------+--------------------+-----------+



In [15]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
# Start the timer
start_time = time.time()

# Query to calculate the average price of a home per "view" rating using the Parquet data
avg_price_per_view_parquet = spark.sql("""
    SELECT
        view AS view_rating,
        ROUND(AVG(price), 2) AS avg_price
    FROM partitioned_home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view_rating DESC
""")

# Show the results
avg_price_per_view_parquet.show()

# End the timer and print the runtime
print("--- %s seconds ---" % (time.time() - start_time))


+-----------+----------+
|view_rating| avg_price|
+-----------+----------+
|        100| 1026669.5|
|         99|1061201.42|
|         98|1053739.33|
|         97|1129040.15|
|         96|1017815.92|
|         95| 1054325.6|
|         94| 1033536.2|
|         93|1026006.06|
|         92| 970402.55|
|         91|1137372.73|
|         90|1062654.16|
|         89|1107839.15|
|         88|1031719.35|
|         87| 1072285.2|
|         86|1070444.25|
|         85|1056336.74|
|         84|1117233.13|
|         83|1033965.93|
|         82| 1063498.0|
|         81|1053472.79|
+-----------+----------+
only showing top 20 rows

--- 1.0793907642364502 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.

spark.sql("UNCACHE TABLE home_sales")

# Verify that the table is uncached
print("Is 'home_sales' table cached? ", spark.catalog.isCached("home_sales"))


Is 'home_sales' table cached?  False


In [17]:
# 15. Check if the home_sales is no longer cached

is_cached = spark.catalog.isCached("home_sales")

# Print the result
print(f"Is 'home_sales' table cached? {is_cached}")


Is 'home_sales' table cached? False
